# Neural Network Demo

## Loading Data

We'll demo this Vanilla Neural Network on MNIST Data. First things first, we want to load our MNIST Data

In [1]:
from DataLoader import *

# load up data
loader = DataLoader()
loader.load_trainval()
loader.load_test()

# retrieve loaded data
train = loader.get_train()
val = loader.get_val()
test = loader.get_test()

# test by printing first training image
row1 = train['data'][0]
for row in range(28):
    for col in range(28):
        print(' ' if row1[28*row+col] == 0 else '+', end='')
    print('')

                            
                            
                            
                            
                            
            ++++++++++++    
        ++++++++++++++++    
       ++++++++++++++++     
       +++++++++++          
        +++++++ ++          
         +++++              
           ++++             
           ++++             
            ++++++          
             ++++++         
              ++++++        
               +++++        
                 ++++       
              +++++++       
            ++++++++        
          +++++++++         
        ++++++++++          
      ++++++++++            
    ++++++++++              
    ++++++++                
                            
                            
                            


## Building the Neural Network

Now that we have our data, we can build a simple vanila neural network.

In [2]:
from Net import *

num_inputs = 784
num_outputs = 10

net = Net([num_inputs, 500, 250, 50, num_outputs])

Initializing weights... Done


## Training the Neural Network on SGD

We'll start of by demoing and testing SGD. We'll build a train function, applying the SGD optimizer. For now, we'll just use a batch size of 1.

In [3]:
from Optimizers import *

loss = []

def train_net(epochs, lr, decay):
    sgd = SGD(lr, decay)
    
    for epoch in range(epochs):
        print('Epoch ' + str(epoch) + '...')
        
        avg_loss = 0
        for entry in range(len(train['data'])):
            net(train['data'][entry])
        
            grad, upstreams = net.backward(train['label'][entry])
            for n in range(len(net.layers) - 2, -1, -1):
                if isinstance(net.layers[n], Linear):
                    net.layers[n].update(sgd)
            
            sgd.advance()
            
            curr_loss = net.xent(train['label'][entry])
            avg_loss += curr_loss
            if entry % 20 == 0:
                loss.append(curr_loss)
            
        avg_loss /= len(train['data'])
            
        print('Loss for Epoch ' + str(epoch) + ': ' + str(avg_loss))

In [4]:
num_epoch = 10
lr = 0.001
decay = 1e-3

train_net(num_epoch, lr, decay)

Epoch 0...
Loss for Epoch 0: 1.2586704171099259
Epoch 1...
Loss for Epoch 1: 1.2538369999515555
Epoch 2...
Loss for Epoch 2: 1.2538369999515555
Epoch 3...
Loss for Epoch 3: 1.2538369999515555
Epoch 4...
Loss for Epoch 4: 1.2538369999515555
Epoch 5...
Loss for Epoch 5: 1.2538369999515555
Epoch 6...


KeyboardInterrupt: 

In [5]:
val_accuracy = 0

for entry in range(len(val['data'])):
    pred = np.argmax(net(val['data'][entry]))
    if pred == val['label'][entry]:
        val_accuracy += 1
        
val_accuracy /= len(val['data'])
print('Validation Accuracy: ' + str(val_accuracy))

Validation Accuracy: 0.74365


In [6]:
test_accuracy = 0

for entry in range(len(test['data'])):
    pred = np.argmax(net(test['data'][entry]))
    if pred == test['label'][entry]:
        test_accuracy += 1
        
test_accuracy /= len(test['data'])
print('Test Accuracy: ' + str(test_accuracy))

Test Accuracy: 0.7407


In [9]:
test_data = test['data']
n = len(test_data)

for k in range(0, 3):
    for i in range(28):
        for j in range(28):
            print(' ' if test_data[k][28*i+j] == 0 else '+', end='')
        print()

    print('Label: ' + str(np.argmax(net(test_data[k]))))

                            
                            
                            
                            
                            
                            
                            
      ++++++                
      ++++++++++++++++      
      ++++++++++++++++      
           +++++++++++      
                  ++++      
                 ++++       
                 ++++       
                ++++        
                ++++        
               ++++         
               +++          
              ++++          
             ++++           
            +++++           
            ++++            
           +++++            
           ++++             
          +++++             
          +++++             
          ++++              
                            
Label: 7
                            
                            
                            
          +++++++           
         +++++++++          
        ++++++++++          
     

#### Learning Curve
Since we have a set of losses over our iterations, we can look at and analyze a learning curve.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss)
plt.show()